# Something about Board Games bruh

## Scraping the data

In [ ]:
# %%writefile spider.py
import re

from bs4 import BeautifulSoup

from scrapy.spiders import CrawlSpider
from scrapy.item import Item, Field
from scrapy.crawler import CrawlerProcess


class Game(Item):
    id = Field()
    title = Field()
    geek_rate = Field()
    avg_rate = Field()
    num_votes = Field()

class GameSpider(CrawlSpider):
    name = 'boardgamegeek_spider'
    allowed_domains = ['boardgamegeek.com']
    start_urls = ['http://www.boardgamegeek.com/browse/boardgame']
    custom_settings = {'DOWNLOAD_DELAY': 1, 'BOT_NAME': 'Mr. Scraper Bot'}

    def parse(self, response):
        soup = BeautifulSoup(response.body, 'lxml')
        next_page = soup.select('a[title^="next"]')
        if next_page:
            next_page = next_page[0].get('href')
        if next_page:
            yield self.make_requests_from_url(response.urljoin(next_page))
        rows = soup.select('tr#row_')
        for row in rows:
            g = Game()
            a = row.find_all('a', href=re.compile('^/boardgame'))
            for r in a:
                if r.text:
                    g['id'] = r.get('href').split('/')[2]
                    g['title'] = r.text
            geek_rate, avg_rate, num_votes = row.select('td.collection_bggrating')
            g['geek_rate'] = geek_rate.text.strip()
            g['avg_rate'] = avg_rate.text.strip()
            g['num_votes'] = num_votes.text.strip()
            yield g

scrapy runspider spider.py -o items.csv

In [24]:
import csv

filename = 'items.csv'
reader = csv.reader(open(filename))
col = next(reader).index('id')
ids = [line[col] for line in reader]
ids = list(set(ids))
with open('ids.txt', 'w') as f:
    f.write('\n'.join(ids))

In [77]:
import csv
import time
import requests
from bs4 import BeautifulSoup

def get_val(tag, term):
    try:
        val = tag.find(term)['value'].encode('utf_8', 'ignore')
    except:
        val = 'NaN'
    return val

base = 'http://www.boardgamegeek.com/xmlapi2/thing?id={}&stats=1'
with open('ids.txt') as f:
    ids = [line.strip() for line in f.readlines()]
split = 30
f = open('games.csv', 'w', encoding='utf8')
writer = csv.writer(f)
writer.writerow(('id', 'type', 'name', 'yearpublished', 'minplayers', 'maxplayers', 'playingtime',
                 'minplaytime', 'maxplaytime', 'minage', 'users_rated', 'average_rating',
                 'bayes_average_rating', 'total_owners', 'total_traders', 'total_wanters',
                 'total_wishers', 'total_comments', 'total_weights', 'average_weight'))
for i in range(0, len(ids), split):
    url = base.format(','.join(ids[i:i+split]))
    print('Requesting {}'.format(url))
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'xml')
    items = soup.find_all('item')
    for item in items:
        gid = item['id']
        gtype = item['type']
        gname = get_val(item, 'name')
        gyear = get_val(item, 'yearpublished')
        gmin = get_val(item, 'minplayers')
        gmax = get_val(item, 'maxplayers')
        gplay = get_val(item, 'playingtime')
        gminplay = get_val(item, 'minplaytime')
        gmaxplay = get_val(item, 'maxplaytime')
        gminage = get_val(item, 'minage')
        usersrated = get_val(item.statistics.ratings, 'usersrated')
        avg = get_val(item.statistics.ratings, 'average')
        bayesavg = get_val(item.statistics.ratings, 'bayesaverage')
        owners = get_val(item.statistics.ratings, 'owned')
        traders = get_val(item.statistics.ratings, 'trading')
        wanters = get_val(item.statistics.ratings, 'wanting')
        wishers = get_val(item.statistics.ratings, 'wishing')
        numcomments = get_val(item.statistics.ratings, 'numcomments')
        numweights = get_val(item.statistics.ratings, 'numweights')
        avgweight = get_val(item.statistics.ratings, 'averageweight')
        writer.writerow((gid, gtype, gname.decode(), gyear.decode(), gmin.decode(), gmax.decode(),
                         gplay.decode(), gminplay.decode(), gmaxplay.decode(), gminage.decode(),
                         usersrated.decode(), avg.decode(), bayesavg.decode(), owners.decode(),
                         traders.decode(), wanters.decode(), wishers.decode(), numcomments.decode(),
                         numweights.decode(), avgweight.decode()))
    time.sleep(2)
f.close()

Requesting http://www.boardgamegeek.com/xmlapi2/thing?id=123505,40141,12715,140974,26232,2771,184520,201557,155526,38719,186715,23231,24762,76544,66351,169352,165762,64603,179718,4725,122838,66153,182300,60294,62216,163030,195210,203068,104640,20189&stats=1
Requesting http://www.boardgamegeek.com/xmlapi2/thing?id=214822,189016,1701,181476,186271,143661,162543,114202,137497,14959,36993,61488,197152,11284,92363,116097,133978,151952,209774,23613,15796,6616,19814,33010,108792,30961,38627,3956,60741,8295&stats=1
Requesting http://www.boardgamegeek.com/xmlapi2/thing?id=20577,665,2576,208437,163243,46659,162541,11316,133967,17259,36758,120977,52926,25711,209393,30501,22821,166418,23831,128659,11186,1290,23871,91365,31036,8703,131014,100115,143296,24827&stats=1
Requesting http://www.boardgamegeek.com/xmlapi2/thing?id=73538,202177,28882,6101,2897,187987,30922,32738,2793,111957,211180,197977,130751,7792,35450,15714,194626,25926,39201,24760,61216,466,19563,3508,66666,128771,18500,162476,36115,122

KeyboardInterrupt: 

In [8]:
import pandas as pd
import numpy as np
games = pd.read_csv('games.csv')
games = games[games['users_rated'] > 30]
games.head()

,id,type,name,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,users_rated,average_rating,bayes_average_rating,total_owners,total_traders,total_wanters,total_wishers,total_comments,total_weights,average_weight
2,12715,boardgame,De Bellis Renationis,1997.0,1.0,2.0,180.0,180.0,180.0,14.0,47,6.59894,5.55511,117,2,0,4,16,9,3.5556
5,2771,boardgame,Maginor,2001.0,2.0,4.0,45.0,45.0,45.0,12.0,328,5.49909,5.49931,739,78,3,14,144,38,1.8684
12,24762,boardgame,Medici vs Strozzi,2006.0,2.0,2.0,30.0,30.0,30.0,10.0,1538,6.52968,6.14922,1808,115,53,205,676,194,2.4485
28,104640,boardgame,Robin Hood,2011.0,3.0,5.0,50.0,50.0,50.0,14.0,182,7.15920,5.77207,251,12,49,120,48,17,1.4706
32,1701,boardgame,Toscana,2001.0,2.0,2.0,20.0,20.0,20.0,10.0,320,5.64484,5.54341,571,53,11,19,106,26,1.5385
